<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw2part1_johnbarry_20200120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 part 1
## CS 590.07
## John Barry





## Load in simlex999 data from github

In [0]:
import pandas as pd
from random import randint



In [0]:
url = 'https://raw.githubusercontent.com/jwb4335/nlp/master/Sim_Lex999.txt'
simlex = pd.read_csv(url,sep = ':')
simlex.columns = ['word1','word2','SimLex999','POS1','POS2']



```
# This is formatted as code
```

Let's get three random rows

In [105]:
randints = [randint(0,len(simlex)), randint(0,len(simlex)),randint(0,len(simlex))]

simlex_rand = simlex.iloc[randints,:]
simlex_rand



,word1,word2,SimLex999,POS1,POS2
568,value,belief,7.07,n,n
492,oil,mink,1.23,n,n
478,attitude,confidence,4.35,n,n


## **Universal sentence encoder** 





In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [109]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



In [110]:
message_embedding_snippet

'-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904'

In [0]:
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

In [0]:
message_embeddings

# CBOW or word2index

Download the data

In [126]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-21 03:16:48--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt.2’

reviews.txt.2       100%[===================>]  32.12M  43.5MB/s    in 0.7s    

2020-01-21 03:16:49 (43.5 MB/s) - ‘reviews.txt.2’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of th

In [127]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-21 03:16:54--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt.2’

labels.txt.2        100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-01-21 03:16:54 (4.61 MB/s) - ‘labels.txt.2’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [129]:
vocab[11]

'mockumentaries'

In [130]:
vocab

['quaresma',
 'governments',
 'sounding',
 'ager',
 'b',
 'heywood',
 'birthday',
 'as',
 'ps',
 'collar',
 'hamper',
 'mockumentaries',
 'waxed',
 'colera',
 'coe',
 'reroutes',
 'nabiki',
 'raspberry',
 'hoarsely',
 'homoeroticism',
 'showstoppingly',
 'mountainbillies',
 'khialdi',
 'mccartney',
 'clint',
 'hernand',
 'fhrer',
 'lisette',
 'labeija',
 'emit',
 'squatting',
 'halle',
 'nicalo',
 'cortland',
 'respectable',
 'baptised',
 'obtained',
 'klinghoffer',
 'gibbs',
 'boldt',
 'carrer',
 'attacker',
 'hendler',
 'yesteryears',
 'superchick',
 'transience',
 'gautham',
 'kellaway',
 'provision',
 'exclamations',
 'disintegrates',
 'feeling',
 'chrissy',
 'inhi',
 'ug',
 'multitudinous',
 'mist',
 'miyaan',
 'vays',
 'grandiose',
 'rodman',
 'molinaro',
 'thong',
 'sportsman',
 'godparents',
 'invincible',
 'naoto',
 'dowagers',
 'excelsior',
 'sputnick',
 'outranks',
 'snowmonton',
 'infinite',
 'ascertain',
 'foppery',
 'bytes',
 'musts',
 'kalisher',
 'overheated',
 'agonize

Initialize Weights

In [131]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]


Run the Model

In [132]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.8329166666666666%
Iter:1 Progress:95.99% Training Accuracy:0.8666041666666666%
Test Accuracy:0.845


Let's get the vector for a word

In [133]:
target = "dentist"
target_index = word2index[target]
weights_0_1[target_index]

array([ 0.06009568,  0.0399876 ,  0.0796992 , -0.09598017,  0.02441107,
       -0.05011182,  0.04864944,  0.01432665, -0.08427631, -0.02424234,
       -0.1110188 , -0.00092907,  0.01382634,  0.04817731,  0.11341218,
        0.0482275 ,  0.02766825, -0.08788557,  0.04285284,  0.01776149,
        0.0470339 , -0.01960356,  0.04736899,  0.04201924,  0.08765869,
        0.01320482, -0.07880731, -0.02327635,  0.07143301,  0.09083226,
       -0.02652866,  0.06213588, -0.0578611 ,  0.06648791,  0.08729845,
        0.02656941, -0.1150595 ,  0.04982296,  0.10603158, -0.08384936,
       -0.02744567,  0.07495822, -0.10328521, -0.06463309, -0.05507632,
        0.04800561, -0.09533671,  0.06146645,  0.01385028,  0.08964772,
       -0.06842212, -0.05617257,  0.0642133 , -0.0538156 , -0.05942922,
       -0.0207913 ,  0.02416927, -0.00484816,  0.05358356, -0.01649914,
       -0.0676014 ,  0.10474029,  0.0844068 ,  0.05695092,  0.03845371,
        0.04076037, -0.0437208 , -0.04707228, -0.05966761,  0.01

# Word2vec

In [134]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-21 03:17:25--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M  2.17MB/s    in 14s     

2020-01-21 03:17:39 (2.14 MB/s) - ‘text8.zip’ saved [31344016/31344016]



In [135]:
!unzip text8.zip

Archive:  text8.zip
replace text8? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: text8                   


In [136]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [139]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8',size=100, window=5, min_count=1, workers=4)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's get the vector for a word

In [140]:
vector = model.wv['computer']
print(vector)

[ 0.11289718 -0.04422016  0.00180758 -0.05244258  0.04945259  0.05354308
 -0.06185471 -0.01307127 -0.0034565   0.0579153   0.01254798  0.02892429
 -0.11332789  0.03532086 -0.09531708 -0.029345   -0.07718465 -0.01858856
 -0.07120853  0.07782654 -0.03414229 -0.14115459  0.09128417 -0.05705065
 -0.07077157  0.0075607   0.03936555  0.04517461  0.07232532 -0.01993036
  0.02286101  0.00512438 -0.0308705  -0.08631044  0.00925408  0.02353635
  0.09170044  0.10479543 -0.05660923  0.00542756 -0.04016088  0.08971819
 -0.10769056  0.02799187  0.03116529 -0.00315698  0.07421315 -0.02154413
 -0.00978852  0.04468653 -0.00960887  0.0692682   0.0433637   0.06471037
 -0.05130323 -0.07337978 -0.0917681  -0.06012401  0.06624223  0.05527163
 -0.00722482  0.13473813  0.02162848 -0.0002016  -0.02273206  0.08307061
  0.07453457 -0.07231398  0.00644889  0.02980266 -0.02262798 -0.07921049
  0.08299242  0.1355353   0.0235066  -0.05111807 -0.07920542 -0.05046162
  0.04100739 -0.0323855  -0.08218705 -0.06942842 -0

## Get the similarity scores of the three embeddings

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [142]:
pd.options.mode.chained_assignment = None  # default='warn'
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
  print('Running universal sentence encoder on "{}"'.format(word1))
  messages = [word1]
  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))
      print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

  vector1 = message_embedding
  vector1 = np.asarray(vector1)
  vector1 = vector1.reshape(1,512)

  print('Running universal sentence encoder on "{}"'.format(word2))
  messages = [word2]

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

  vector2 = message_embedding
  vector2 = np.asarray(vector2)
  vector2 = vector2.reshape(1,512)

  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
  simlex_rand.loc[simlex_rand.index == num,'USE Cosine Sim'] = cos_sim

Embedding this pair from Simlex_999: "value" and "belief"
Running universal sentence encoder on "value"
Message: value
Embedding size: 512
Embedding: [0.023315245285630226, 0.002574664307758212, -0.04323190823197365, ...]

Running universal sentence encoder on "belief"
Message: belief
Embedding size: 512
Cosine similarity from Universal sentence encoder for value and belief = [[0.26116359]]
Embedding this pair from Simlex_999: "oil" and "mink"
Running universal sentence encoder on "oil"
Message: oil
Embedding size: 512
Embedding: [-0.022792192175984383, 0.05218827724456787, -0.009384042583405972, ...]

Running universal sentence encoder on "mink"
Message: mink
Embedding size: 512
Cosine similarity from Universal sentence encoder for oil and mink = [[0.31359772]]
Embedding this pair from Simlex_999: "attitude" and "confidence"
Running universal sentence encoder on "attitude"
Message: attitude
Embedding size: 512
Embedding: [-0.014253339730203152, -0.0031567823607474566, -0.0322922766208

Embedding method 2

In [143]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  target_index1 = word2index[word1]
  target_index2 = word2index[word2]
  vector1 = weights_0_1[target_index1]
  vector1 = vector1.reshape(1,100)
  vector2 = weights_0_1[target_index2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2index Cosine Sim'] = cos_sim


running word2index on the pair value and belief
Cosine similarity from word2index for value and belief = [[0.0463035]]
running word2index on the pair oil and mink
Cosine similarity from word2index for oil and mink = [[0.05352225]]
running word2index on the pair attitude and confidence
Cosine similarity from word2index for attitude and confidence = [[-0.15378198]]


Embedding method word2vec



In [145]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2vec on the pair {} and {}'.format(word1,word2))
  vector1 = model.wv[word1]
  vector1 = vector1.reshape(1,100)
  vector2 = model.wv[word2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2vec Cosine Sim'] = cos_sim



running word2vec on the pair value and belief
Cosine similarity from word2vec for value and belief = [[0.974757]]
running word2vec on the pair oil and mink
Cosine similarity from word2vec for oil and mink = [[0.95331794]]
running word2vec on the pair attitude and confidence
Cosine similarity from word2vec for attitude and confidence = [[0.128643]]


In [154]:
simlex_rand_out = simlex_rand[['word1','word2','SimLex999', 'USE Cosine Sim',
       'word2index Cosine Sim', 'word2vec Cosine Sim']]
simlex_rand_out.columns = ['word1','word2','SimLex999', 'USE',
       'CBOW', 'word2vec']
simlex_rand_out = round(simlex_rand_out,4)
print('The cosine similarities from the 3 word embedding methods can be seen below.\nThe similarity scores are not close to the Simlex999 scores')
print(simlex_rand_out)

The cosine similarities from the 3 word embedding methods can be seen below.
The similarity scores are not close to the Simlex999 scores
        word1       word2  SimLex999     USE    CBOW  word2vec
568     value      belief       7.07  0.2612  0.0463    0.9748
492       oil        mink       1.23  0.3136  0.0535    0.9533
478  attitude  confidence       4.35  0.7212 -0.1538    0.1286
